**IBM Applied Data Science Capstone Course by Coursera**

**Week 5 Final Report**

**Opening a New Shopping Mall in Chennai, India**

    Build a dataframe of neighborhoods in Chennai, India by web scraping the data from Wikipedia page
    Get the geographical coordinates of the neighborhoods
    Obtain the venue data for the neighborhoods from Foursquare API
    Explore and cluster the neighborhoods
    Select the best cluster to open a new shopping mall

## 1.Import Libraries

In [4]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 93 kB 3.9 MB/s  eta 0:00:01
Libraries imported.


## 2.Scrap data from Wikipedia page into a DataFrame

In [6]:

# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Chennai").text

In [7]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [8]:
# create a list to store neighborhood data
neighborhoodList = []

In [9]:

# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [14]:
# create a new DataFrame from the list
ch_df = pd.DataFrame({"Neighborhood": neighborhoodList})

ch_df.head()

,Neighborhood
0,Alandur
1,Anna Nagar
2,"Ashok Nagar, Chennai"
3,Assisi Nagar
4,Avadi


In [15]:

# print the number of rows of the dataframe
ch_df.shape

(67, 1)

## 3.Get the geographical coordinates

In [12]:

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [16]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in ch_df["Neighborhood"].tolist() ]

In [17]:
coords

[[13.00013000000007, 80.20049000000006],
 [13.083590000000072, 80.21015000000006],
 [13.035390000000064, 80.21220000000005],
 [13.164570000000026, 80.23274000000004],
 [13.129550000000052, 80.10356000000007],
 [13.09883000000002, 80.23238000000003],
 [12.932770000000062, 80.14387000000005],
 [12.95237000000003, 80.14413000000008],
 [12.988610000000051, 80.15100000000007],
 [12.82725000000005, 80.22866000000005],
 [12.837960000000066, 80.05317000000008],
 [13.040920000000028, 80.13649000000004],
 [13.110190000000046, 80.21282000000008],
 [13.129720000000077, 80.18300000000005],
 [13.120580000000075, 80.06047000000007],
 [12.956150000000036, 80.17885000000007],
 [12.786140000000046, 80.22030000000007],
 [13.08197000000007, 80.24448000000007],
 [13.051520000000039, 80.22436000000005],
 [13.136630000000025, 80.24479000000008],
 [13.115007966144393, 80.21147930624271],
 [13.096140000000048, 80.05279000000007],
 [13.116800000000069, 80.27726000000007],
 [13.183260000000075, 80.24059000000005

In [18]:

# create temporary dataframe to populate the coordinates into Latitude and Longitude
ch_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [19]:

# merge the coordinates into the original dataframe
ch_df['Latitude'] = ch_coords['Latitude']
ch_df['Longitude'] = ch_coords['Longitude']

In [20]:
# check the neighborhoods and the coordinates
print(ch_df.shape)
ch_df

(67, 3)


,Neighborhood,Latitude,Longitude
0,Alandur,13.000130,80.200490
1,Anna Nagar,13.083590,80.210150
2,"Ashok Nagar, Chennai",13.035390,80.212200
3,Assisi Nagar,13.164570,80.232740
4,Avadi,13.129550,80.103560
5,Ayanavaram,13.098830,80.232380
6,Chitlapakkam,12.932770,80.143870
7,Chromepet,12.952370,80.144130
8,Cowl Bazaar,12.988610,80.151000
9,Egattur (Kanchipuram District),12.827250,80.228660


In [21]:
# save the DataFrame as CSV file
ch_df.to_csv("ch_df.csv", index=False)

### 4. Create a map of Chennai with neighborhoods superimposed on top

In [23]:
# get the coordinates of Chennai
address = 'Chennai, India'

geolocator = Nominatim(user_agent="chennai_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chennai, India 13.0836939, 80.270186.


In [24]:

# create map of Chennai using latitude and longitude values
map_ch = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(ch_df['Latitude'], ch_df['Longitude'], ch_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ch)  
    
map_ch

In [26]:

# save the map as HTML file
map_ch.save('map_ch.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [57]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: AHX1DW2HX35QO2BNUEFL44FBCO4YZVSZPWNT24E4MGNGURYJ
CLIENT_SECRET:XYPHP1AX5U2KKCOQOU4T042RZLXOMX22STSIZZXV4DATNJZY


### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [28]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(ch_df['Latitude'], ch_df['Longitude'], ch_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [29]:

# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1196, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alandur,13.00013,80.20049,Sukkkubai Beef Biryani Shop,12.998769,80.201381,Indian Restaurant
1,Alandur,13.00013,80.20049,Pizza Republic,12.990987,80.198613,Pizza Place
2,Alandur,13.00013,80.20049,Moon & Six Pence - The Irish Bar,13.007848,80.208152,Bar
3,Alandur,13.00013,80.20049,Q Bar,13.016606,80.204853,Restaurant
4,Alandur,13.00013,80.20049,St Thomas Mount Railway Station,12.994987,80.200302,Train Station


### Let's check how many venues were returned for each neighborhood

In [30]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alandur,44,44,44,44,44,44
Anna Nagar,100,100,100,100,100,100
"Ashok Nagar, Chennai",67,67,67,67,67,67
Assisi Nagar,3,3,3,3,3,3
Avadi,8,8,8,8,8,8
Ayanavaram,30,30,30,30,30,30
Chitlapakkam,16,16,16,16,16,16
Chromepet,19,19,19,19,19,19
Cowl Bazaar,21,21,21,21,21,21


#### Let's find out how many unique categories can be curated from all the returned venues

In [31]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 149 uniques categories.


In [32]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Indian Restaurant', 'Pizza Place', 'Bar', 'Restaurant',
       'Train Station', 'Hotel', 'Church', 'Asian Restaurant',
       'Ice Cream Shop', 'Donut Shop', 'South Indian Restaurant',
       'Breakfast Spot', 'Multicuisine Indian Restaurant',
       'Japanese Restaurant', 'Bakery', 'Pool Hall', 'Airport', 'Café',
       'Juice Bar', 'Fast Food Restaurant', 'Metro Station', 'Cafeteria',
       'Golf Course', 'Gym Pool', 'Sandwich Place', 'Park', 'Gym',
       'Snack Place', 'Chinese Restaurant',
       'Vegetarian / Vegan Restaurant', 'Coffee Shop', 'Shoe Store',
       'Indian Sweet Shop', 'American Restaurant', 'Burger Joint',
       'Shopping Mall', 'Middle Eastern Restaurant', 'Italian Restaurant',
       'Department Store', 'Clothing Store', 'New American Restaurant',
       'BBQ Joint', 'Paper / Office Supplies Store', 'Market',
       'Furniture / Home Store', 'Multiplex', 'Farmers Market',
       'Jewelry Store', 'Bookstore', 'Bistro'], dtype=object)

In [34]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each neighborhood

In [35]:
# one hot encoding
ch_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ch_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ch_onehot.columns[-1]] + list(ch_onehot.columns[:-1])
ch_onehot = ch_onehot[fixed_columns]

print(ch_onehot.shape)
ch_onehot.head()

(1196, 150)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Andhra Restaurant,Asian Restaurant,Auto Workshop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Buffet,Burger Joint,Bus Line,Bus Station,Cafeteria,Café,Campground,Canal,Chinese Restaurant,Church,Clothing Store,Coffee Shop,College Cafeteria,Concert Hall,Convenience Store,Coworking Space,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Duty-free Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,Hospital,Hotel,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,North Indian Restaurant,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Platform,Playground,Pool Hall,Pub,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,River,Road,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Club,Supermarket,Taxi Stand,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Zoo,Zoo Exhibit
0,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [36]:
ch_grouped = ch_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(ch_grouped.shape)
ch_grouped

(66, 150)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Andhra Restaurant,Asian Restaurant,Auto Workshop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Buffet,Burger Joint,Bus Line,Bus Station,Cafeteria,Café,Campground,Canal,Chinese Restaurant,Church,Clothing Store,Coffee Shop,College Cafeteria,Concert Hall,Convenience Store,Coworking Space,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Duty-free Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,Hospital,Hotel,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,North Indian Restaurant,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Platform,Playground,Pool Hall,Pub,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,River,Road,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Club,Supermarket,Taxi Stand,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Zoo,Zoo Exhibit
0,Alandur,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.00,0.000000,0.022727,0.0,0.000000,0.000000,0.022727,0.000000,0.045455,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.022727,0.045455,0.000000,0.0,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.022727,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.113636,0.000000,0.000000,0.022727,0.204545,0.00,0.000000,0.000000,0.000000,0.022727,0.00,0.022727,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.045455,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.068182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
1,Anna Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.010000,0.0,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.040000,0.000000,0.0,0.030000,0.000000,0.050000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.010000,0.100000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.030000,0.140000,0.01,0.000000,0.000000,0.010000,0.000000,0.01,0.010000,0.000

In [38]:
len(ch_grouped[ch_grouped["Shopping Mall"] > 0])

11

### Create a new DataFrame for Shopping Mall data only

In [39]:
ch_mall = ch_grouped[["Neighborhoods","Shopping Mall"]]

In [40]:
ch_mall.head()

,Neighborhoods,Shopping Mall
0,Alandur,0.000000
1,Anna Nagar,0.020000
2,"Ashok Nagar, Chennai",0.014925
3,Assisi Nagar,0.000000
4,Avadi,0.125000


### Cluster Neighborhood

Run k-means to cluster the neighborhoods in Chennai into 3 clusters.

In [41]:
# set number of clusters
kclusters = 3

ch_clustering = ch_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ch_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 2, 1, 0, 1, 0, 0], dtype=int32)

In [43]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
ch_merged = ch_mall.copy()

# add clustering labels
ch_merged["Cluster Labels"] = kmeans.labels_

In [44]:
ch_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
ch_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alandur,0.000000,0
1,Anna Nagar,0.020000,0
2,"Ashok Nagar, Chennai",0.014925,0
3,Assisi Nagar,0.000000,0
4,Avadi,0.125000,2


In [45]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ch_merged = ch_merged.join(ch_df.set_index("Neighborhood"), on="Neighborhood")

print(ch_merged.shape)
ch_merged.head() # check the last columns!

(66, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alandur,0.000000,0,13.00013,80.20049
1,Anna Nagar,0.020000,0,13.08359,80.21015
2,"Ashok Nagar, Chennai",0.014925,0,13.03539,80.21220
3,Assisi Nagar,0.000000,0,13.16457,80.23274
4,Avadi,0.125000,2,13.12955,80.10356


In [47]:
# sort the results by Cluster Labels
print(ch_merged.shape)
ch_merged.sort_values(["Cluster Labels"], inplace=True)
ch_merged

(66, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alandur,0.000000,0,13.000130,80.200490
34,Oragadam,0.000000,0,13.137440,80.153830
35,Padappai,0.000000,0,12.877076,80.048448
36,Palavakkam,0.000000,0,12.961400,80.257440
37,Pallavaram,0.000000,0,12.974440,80.148520
38,Pallikaranai,0.000000,0,12.955670,80.220800
40,Panambakkam,0.000000,0,13.077620,80.155840
41,Pattabiram,0.000000,0,13.123330,80.059440
42,Peerkankaranai,0.000000,0,12.912240,80.098950
43,Perambakkam,0.000000,0,12.905630,80.209070


### Finally, let's visualize the resulting clusters

In [49]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ch_merged['Latitude'], ch_merged['Longitude'], ch_merged['Neighborhood'], ch_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [50]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8.Examine Clusters

Cluster 0

In [54]:
ch_merged.loc[ch_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alandur,0.000000,0,13.000130,80.200490
34,Oragadam,0.000000,0,13.137440,80.153830
35,Padappai,0.000000,0,12.877076,80.048448
36,Palavakkam,0.000000,0,12.961400,80.257440
37,Pallavaram,0.000000,0,12.974440,80.148520
38,Pallikaranai,0.000000,0,12.955670,80.220800
40,Panambakkam,0.000000,0,13.077620,80.155840
41,Pattabiram,0.000000,0,13.123330,80.059440
42,Peerkankaranai,0.000000,0,12.912240,80.098950
43,Perambakkam,0.000000,0,12.905630,80.209070


Cluster 1

In [55]:
ch_merged.loc[ch_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
39,Pammal,0.040000,1,12.96817,80.13358
7,Chromepet,0.052632,1,12.95237,80.14413
5,Ayanavaram,0.033333,1,13.09883,80.23238
32,Navalur,0.062500,1,12.84584,80.22648


Cluster 2

In [56]:
ch_merged.loc[ch_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
4,Avadi,0.125,2,13.12955,80.10356


**Observations:**

Most of the shopping malls are concentrated in Cluster 2 which is the northern side of Chennai, especially in 'Avadi' neighborhood. Cluster 1 have moderate number of Shopping Malls which comprise of 4 neighborhoods - Pammal, Chromper, Ayanavaram and Navalur. On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the northern side of the city, with the southern area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.